**Dataset**
labeled datasset collected from Spotify (Assignment 1 - Spotify Reviews Rating)

**Objective**
classify Review to a category from 1 to 5. <br>

**Total Estimated Time = 90-120 Mins**

**Evaluation metric**
macro f1 score

### Import used libraries

In [ ]:
import numpy as np
import pandas as pd
import re
import string
import spacy
import contractions
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from gensim.models import KeyedVectors
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Flatten, Dense, GlobalAveragePooling1D,Dropout
! python -m spacy download en_core_web_md

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Load Dataset

In [4]:
df = pd.read_csv("Assignment 1 - Spotify Reviews Rating.csv")

In [5]:
pd.set_option('display.max_rows',500)
pd.set_option('display.max_colwidth',500)

In [ ]:
df.head(50)

,Time_submitted,Review,Rating
0,7/9/2022 15:00,"Great music service, the audio is high quality and the app is easy to use. Also very quick and friendly support.",5
1,7/9/2022 14:21,Please ignore previous negative rating. This app is super great. I give it five stars+,5
2,7/9/2022 13:27,"This pop-up ""Get the best Spotify experience on Android 12"" is too annoying. Please let's get rid of this.",4
3,7/9/2022 13:26,Really buggy and terrible to use as of recently,1
4,7/9/2022 13:20,Dear Spotify why do I get songs that I didn't put on my playlist??? And why do we have shuffle play?,1
5,7/9/2022 13:20,The player controls sometimes disappear for no reason. App restart forgets what I was playing but fixes the issue.,3
6,7/9/2022 13:19,I love the selection and the lyrics are provided with the song you're listening to!,5
7,7/9/2022 13:17,"Still extremely slow when changing storage to external sd card.. I'm convinced this is done on purpose, spotify knows of this issue and has done NOTHING to solve it! Over time I have changed sd cards, each being faster in read, write speeds(all samsung brand). And please add ""don't like song"" so it will never appear again in my searches or playlists.",3
8,7/9/2022 13:16,"It's a great app and the best mp3 music app I have ever used but there is one problem that, why can't we play some songs or find some songs? despite this the app is wonderful I recommend it. it's just the best.",5
9,7/9/2022 13:11,"I'm deleting this app, for the following reasons: This app now has a failing business model. Whether streaming services like it, or not: the consumer doesn't want to pay for music they can't fully own, and 6 ads successively, upon logging in, before a single song, is too much. Closed the app during ad number 6, and I'm more patient than most. If those are the only ways you can profit: you've already peaked. All that's left is your decline.",1


### Data splitting

It is a good practice to split the data before EDA helps maintain the integrity of the machine learning process, prevents data leakage, simulates real-world scenarios more accurately, and ensures reliable model performance evaluation on unseen data.

In [6]:
x = df.drop(['Time_submitted','Rating'],axis=1)
y = df['Rating']

In [7]:
print("shape of x ",x.shape)
print("shape of y",y.shape)

shape of x  (61594, 1)
shape of y (61594,)


In [8]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.15,random_state=42,shuffle=True)

In [9]:
x_train.shape

(52354, 1)

In [10]:
y_train.shape

(52354,)

### EDA on training data

- check NaNs

In [11]:
x_train.isnull().sum()

Review    0
dtype: int64

- check duplicates

In [12]:
duplicate_rows = x_train[x_train.duplicated()]
print(len(duplicate_rows))

181


In [ ]:
x_train[x_train.duplicated()]

,Review
3422,Great app tons of music
18967,This is the best music app
6925,Best music streaming app
2495,Good app for songs
28167,Great variety of music
15697,Best app for music listening
18264,Too much ads
1940,Great music collection
4651,Great selection of music and podcasts
27375,Too many ads


- show a representative sample of data texts to find out required preprocessing steps

In [ ]:
x_train.sample()

,Review
11392,Quit playing songs that are not in my playlist


In [ ]:
x_train.tail(20)

,Review
769,"Been a great experience, highly recommend."
1685,I did love using Spotify! Not anymore since I'm ANNOYED that ALL of my playlists are gone for some reason...
41090,This app is always best....but recently it added a new feature translation i really loved it...it's the best feature so far...when listen to different language songs it's really helps....thank you Spotify for adding this fiture
16023,"Imagine a music streaming app dropping in quality every update it receives. As a user who dosen't spend money on spotify, the experience is pretty bad. The amount of unwanted songs you get is shocking. Even after using up all my skips to get the song that I want to listen to, it still continues to play the unwanted song"
60263,"Generally good, but there's an issue where the file size starts to expand that can only be fixed by uninstalling and reinstalling the app. I listen to Spotify A LOT so it leaves me doing this two to three times a week or having a 3 to 6 GB app. It just gets old."
44131,"Worth listening to my favorite music such as Japanese, korean and Chinese or Hispanic ones. My genre is rock, classic,, metal, pop, while on my school bus and not listening to R&B music... I have nothing against it, it's just not for my ears, I'm probably tone deaf. Ads are bit too much but I can tolerate it as long I can listen to free as much as I can without paying much. (I'M OVER REACTING)~☆!"
60788,Solid app. Rarely crashes. Great features. Wish the free version wasn't shuffled
47191,"Pausing on it's own and not showing the correct song/podcast playing. Awful layout of library. Need a function to delete the unwanted 'your episodes' podcast folder (which annoyingly says 1 episode is in it, when it's empty) and a function to select multiple songs to remove from a playlist instead of one by one. plays a radio playlist based on my playlist I'm listening to, randomly and unwanted. Edit: above not changed, now crashes regularly. Now logged me out on its own and can't log back in"
21962,lags a lot sometimes
37194,In the past few days the app continues to glitch to where I cannot pause my music or skip songs and sleep timers aren't working. I spent money on this and I'm getting backhanded for my own loyalty


- check dataset balancing

In [14]:
y_train.value_counts()

Rating
5    18738
1    14984
4     6705
2     6073
3     5854
Name: count, dtype: int64

- Cleaning and Preprocessing are:
    - 1) Remove duplicated
    - 2) Lowercase
    - 3) Remove digits
    - 4) Remove URLs, mentions, and hashtags
    - 5) Remove non-ASCII characters and special symbols
    - 6) Remove contractions
    - 7) Remove punctuation marks
    - 8) lemmatize text and remove stopwords
    - 9) Word Embedding

### Cleaning and Preprocessing(x_trian)

In [13]:
#Remove duplicated from train data
x_train.drop_duplicates(inplace=True)

y_train.drop(duplicate_rows.index,inplace=True)

In [14]:
x_train.duplicated().sum()

0

In [15]:
x_train.shape , y_train.shape

((52173, 1), (52173,))

In [16]:
#Lowercase
x_train['Review'] = x_train['Review'].apply(lambda x: x.lower())

In [ ]:
x_train.head(50)

,Review
15856,i love listening to spotify
56381,it is nice with buddies of songs..but only need to make it ad-free...it consist 2 adds of 30 seconds each...
48134,aside from live recordings spotify is my go to music app.
56021,i like that they added the lyrics to most of the songs.
40707,outstanding link. free music. your choice. i'm nn
41026,"this app is great as always! i do have one complaint tho. whenever i logout of my account, the lyrics of the songs doesn't appear so i have to close the app and open it again to fix the problem! other than that, keep up the good work :) edit: my problem has been fixed, thank you so much. y'all are my favs!"
34279,"used to love the app, but now crashes on every device i own except my pc. looks like they are just trying to do too much with it. 3 versions now all crash, like others stated, while using both pixel and samsung devices."
20705,"i've used spotify for years now, and on the mobile app there's a bug that causes an ad to play after every single song, i don't know what's causing it. it's annoying cause it ruins my music vibe. i know i can pay for premium, but i should be able to listen to music with an ad here and there but not after every single song."
28765,"spotify is a must have for music enthusiast, professionals and those who want to experience the true gift of sound."
41105,"it's bad service, they don't maintain good quality of podcasts, they don't have independent comments section so that we can give feedback or atleast the artist will improve in their next post."


In [17]:
#remove digits
x_train['Review'] = x_train['Review'].apply(lambda x:re.sub('\d+',"",x))

In [18]:
# Remove URLs, mentions, and hashtags
x_train['Review'] = x_train['Review'].apply(lambda x: re.sub('http\S+|www\S+|@[^\s]+|#\S+',"",x))

In [19]:
# Remove non-ASCII characters and special symbols
x_train['Review'] = x_train['Review'].apply(lambda x: re.sub(r'[^\x00-\x7F]+',"",x))

In [20]:
def preprocess_text(text):
    # Remove contractions
    expanded_text = contractions.fix(text)
    # Remove punctuation marks
    cleaned_text = expanded_text.translate(str.maketrans('', '', string.punctuation))
    return cleaned_text

In [21]:
x_train['Review'] = x_train['Review'].apply(preprocess_text)

In [22]:
stop_words = stopwords.words('english')

In [23]:
lemmatizer = WordNetLemmatizer()

In [24]:
# Function to lemmatize text and remove stopwords
def lemmatize_text(text):
    # Tokenize the text into words
    tokens = text.split()
    # Lemmatize each word and remove stopwords
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens if word.lower() not in stop_words]
    # Join the lemmatized tokens back into a sentence
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text

In [25]:
x_train['Review'] = x_train['Review'].apply(lemmatize_text)

In [ ]:
x_train.head(50)

,Review
15856,love listen spotifi
56381,nice buddi songsbut need make adfreeit consist add second
48134,asid live record spotifi go music app
56021,like ad lyric song
40707,outstand link free music choic nn
41026,app great alway one complaint though whenev logout account lyric song appear close app open fix problem keep good work edit problem fix thank much fav
34279,use love app crash everi devic except pc look like tri much version crash like other state use pixel samsung devic
20705,use spotifi year mobil app bug caus ad play everi singl song know caus annoy ruin music vibe know pay premium abl listen music ad everi singl song
28765,spotifi must music enthusiast profession want experi true gift sound
41105,bad servic maintain good qualiti podcast independ comment section give feedback atleast artist improv next post


**You  are doing Great so far!**

###Cleaning and Preprocessing (Test data)

In [26]:
#Lowercase
x_test['Review'] = x_test['Review'].apply(lambda x: x.lower())

In [27]:
#remove digits
x_test['Review'] = x_test['Review'].apply(lambda x:re.sub('\d+',"",x))

In [28]:
# Remove URLs, mentions, and hashtags
x_test['Review'] = x_test['Review'].apply(lambda x: re.sub('http\S+|www\S+|@[^\s]+|#\S+',"",x))

In [29]:
# Remove non-ASCII characters and special symbols
x_test['Review'] = x_test['Review'].apply(lambda x: re.sub(r'[^\x00-\x7F]+',"",x))

In [30]:
#Remove contractions & Remove punctuation marks
x_test['Review'] = x_test['Review'].apply(preprocess_text)

In [31]:
#Lemmatizer each word
x_test['Review'] = x_test['Review'].apply(lemmatize_text)

In [ ]:
x_test.head(50)

,Review
37326,go spotifi noth play matter fix
50640,music match spot thank spotifi wrote word review year ago need say anyth still go music app everi singl day know would without spotifi like dog moodchang lifesav rock spotifi
27630,lot bug latest version
42850,damn spotifi use amaz stream platform could even play music playlist full length command must shuffl becom money hungri time pass realli want receiv rate
10950,never annoy entir life use applic
26919,spot thank jam tune song help get along bong bong spotifi till fli
32558,app recent give everyon know app matter servic troubl load properli play music download music issu start play give option paus skip play app open song actual play plz help
11787,love spotifi thing wish button click brought music video like youtub
44344,use spotifi sinc mayb later qualm app music found found playlist k count thank metal hard rock metal head could ask
7706,music like one easi navig app


### Word Embedding & Modelling

In [32]:
nlp = spacy.load('en_core_web_md')

In [33]:
vector = nlp("Bank").vector

In [34]:
print(vector.shape)

(300,)


In [35]:
x_train_v = np.zeros((len(x_train['Review']), 300))
x_test_v = np.zeros((len(x_test['Review']), 300))

for i, doc in tqdm(enumerate(nlp.pipe(x_train['Review'])), total=len(x_train['Review'])):
    x_train_v[i, :] = doc.vector

for i, doc in tqdm(enumerate(nlp.pipe(x_test['Review'])), total=len(x_test['Review'])):
    x_test_v[i, :] = doc.vector

100%|██████████| 9240/9240 [00:17<00:00, 513.67it/s]


In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=50, random_state=42)
rf_classifier.fit(x_train_v, y_train)

RandomForestClassifier(n_estimators=50, random_state=42)

In [ ]:
y_pred = rf_classifier.predict(x_test_v)

#### Evaluation

**Evaluation metric:**
macro f1 score

Macro F1 score is a useful metric in scenarios where you want to evaluate the overall performance of a multi-class classification model, **particularly when the classes are imbalanced**

![Calculation](https://assets-global.website-files.com/5d7b77b063a9066d83e1209c/639c3d934e82c1195cdf3c60_macro-f1.webp)

In [ ]:
macro_f1 = f1_score(y_test, y_pred, average='macro')

print("Macro F1-score: {:.2f}".format(macro_f1))

Macro F1-score: 0.31


### Enhancement

### Use Gensim

In [ ]:
! pip install kaggle

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle (1).json
User uploaded file "kaggle (1).json" with length 66 bytes


In [ ]:
! kaggle datasets download -d "leadbest/googlenewsvectorsnegative300"

100% 3.16G/3.17G [00:49<00:00, 146MB/s]
100% 3.17G/3.17G [00:49<00:00, 69.0MB/s]


In [ ]:
! unzip /content/googlenewsvectorsnegative300.zip

Archive:  /content/googlenewsvectorsnegative300.zip
  inflating: GoogleNews-vectors-negative300.bin  
  inflating: GoogleNews-vectors-negative300.bin.gz  


In [ ]:
# Load pre-trained Word2Vec model
word_vectors = KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin.gz', binary=True)


x_train_v = np.zeros((len(x_train['Review']), 300))
x_test_v = np.zeros((len(x_test['Review']), 300))

for i, review in tqdm(enumerate(x_train['Review']), total=len(x_train['Review'])):
    # Tokenize the review and get the average word vector
    words = review.split()
    vectors = [word_vectors[word] for word in words if word in word_vectors]
    if vectors:
        x_train_v[i, :] = np.mean(vectors, axis=0)

for i, review in tqdm(enumerate(x_test['Review']), total=len(x_test['Review'])):
    # Tokenize the review and get the average word vector
    words = review.split()
    vectors = [word_vectors[word] for word in words if word in word_vectors]
    if vectors:
        x_test_v[i, :] = np.mean(vectors, axis=0)


100%|██████████| 9240/9240 [00:00<00:00, 10604.39it/s]


In [ ]:
# Train SVC
svc = SVC(kernel='linear')
svc.fit(x_train_v, y_train)

SVC(kernel='linear')

In [ ]:
y_pred = svc.predict(x_test_v)

In [ ]:
macro_f1 = f1_score(y_test, y_pred, average='macro')

print("Macro F1-score: {:.2f}".format(macro_f1))

Macro F1-score: 0.31


### Use DNN

In [ ]:
x_train_v_reshaped = np.expand_dims(x_train_v, axis=1)

In [ ]:
x_test_v_reshaped = np.expand_dims(x_test_v, axis=1)

In [ ]:
from keras.utils import to_categorical
# One-hot encode the labels
y_train_encoded = to_categorical(y_train-1, num_classes=5)

In [ ]:
model=Sequential()
model.add(GlobalAveragePooling1D(input_shape=(x_train_v_reshaped.shape[1], x_train_v_reshaped.shape[2])))
model.add(Dense(500,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(5,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
history = model.fit(
     x_train_v_reshaped, y_train_encoded,
    epochs=50,
    batch_size=128,

)

Epoch 1/50
408/408 [==============================] - 6s 11ms/step - loss: 1.1368 - accuracy: 0.5649
Epoch 2/50
408/408 [==============================] - 4s 9ms/step - loss: 1.0769 - accuracy: 0.5853
Epoch 3/50
408/408 [==============================] - 4s 10ms/step - loss: 1.0533 - accuracy: 0.5933
Epoch 4/50
408/408 [==============================] - 5s 11ms/step - loss: 1.0334 - accuracy: 0.6008
Epoch 5/50
408/408 [==============================] - 5s 12ms/step - loss: 1.0160 - accuracy: 0.6040
Epoch 6/50
408/408 [==============================] - 8s 20ms/step - loss: 0.9937 - accuracy: 0.6143
Epoch 7/50
408/408 [==============================] - 7s 17ms/step - loss: 0.9723 - accuracy: 0.6215
Epoch 8/50
408/408 [==============================] - 7s 18ms/step - loss: 0.9490 - accuracy: 0.6279
Epoch 9/50
408/408 [==============================] - 5s 13ms/step - loss: 0.9225 - accuracy: 0.6393
Epoch 10/50
408/408 [==============================] - 5s 11ms/step - loss: 0.8967 - accurac

In [ ]:
from sklearn.metrics import accuracy_score
# Make predictions on test set
preds = model.predict(x_test_v_reshaped)

# Retrieve most likely class
class_preds = np.argmax(preds,axis=1)

# Print accuracy
print(f"Test set accuracy: {accuracy_score(y_test, class_preds):.2f}")

289/289 [==============================] - 1s 3ms/step
Test set accuracy: 0.14


### Conclusion and final results


the best solution is used Gensim with linear svc model  or used spicy with rabdom forest model, they are Macro F1-score = 31%

#### Done!